In [1]:
import gzip
from collections import defaultdict
import sklearn
from sklearn import linear_model
import string
import re
import datetime, time
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as md
import math

In [2]:
#####
# Setup
#####

In [3]:
def readCSV(path):
    f = open(path, 'rt')
    f.readline()

    for l in f:
        
        if l.startswith("timestamp"): 
            continue

        try: 
            timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education = re.split(r',(?![ ])', l)

        except:
            continue

        yield timestamp,company,level,title,totalyearlycompensation,location,yearsofexperience,yearsatcompany,tag,basesalary,stockgrantvalue,bonus,gender,otherdetails,cityid,dmaid,rowNumber,Masters_Degree,Bachelors_Degree,Doctorate_Degree,Highschool,Some_College,Race_Asian,Race_White,Race_Two_Or_More,Race_Black,Race_Hispanic,Race,Education

In [4]:
def MSE(predictions, labels):
    differences = [(x-y)**2 for x,y in zip(predictions,labels)]
    return sum(differences) / len(differences)

In [5]:
#####
# Parse data from dataset
#####

In [6]:
# id , unixtime, company, level, title, total_comp, city, state, experience, tenure, tag, 
# salary, stock, bonus, city_id, dma_id, ms_deg, bs_deg, phd_deg, hs, college
def parseSalaryRaw(salaryRaw):
    salaryAllData = []

    for entry in salaryRaw:

        date, clocktime = entry[0].split(' ')
        date = date.split('/')
        clocktime = clocktime.split(':')
        timestamp = datetime.datetime(int(date[2]), int(date[0]), int(date[1]), int(clocktime[0]), int(clocktime[1]), int(clocktime[2]))
        timestamp = int(time.mktime(timestamp.timetuple()))


        company = entry[1]
        level = entry[2]
        title = entry[3]

        total_comp = int(float(entry[4]))

        location = entry[5]
        if location.count(',') == 2:
            city, state, country = location.strip('"').split(', ')
        elif location.count(',') == 1:
            city, state = location.strip('"').split(', ')
            country = 'USA'

        experience = entry[6]
        tenure = entry[7]

        tag = entry[8]

        salary = int(float(entry[9]))
        if salary == 0:
            continue

        stock = int(float(entry[10]))
        bonus = int(float(entry[11]))

        g = entry[12]
        gender = -1
        if g == 'Female':
            gender = 0
        elif g == 'Male':
            gender = 1
        elif g == 'Other':
            gender = 2

        city_id = int(entry[14])

        try: 
            dma_id = int(entry[15])
        except:
            dma_id = -1

        id = int(entry[16])

        ms_deg = int(entry[17])
        bs_deg = int(entry[18])
        phd_deg = int(entry[18])
        hs = int(entry[19])
        college = int(entry[20])

        r = entry[27]
        race = -1
        if r == 'Asian':
            race = 0
        elif r == 'Black':
            race = 1
        elif r == 'Hispanic':
            race = 2
        elif r == 'Two Or More':
            race = 3
        elif r == 'White':
            race = 4


        salaryAllData.append({
            'id': id,
            'timestamp': timestamp,
            'company': company,
            'level': level,
            'title': title,
            'total_comp': total_comp,
            'city': city,
            'state': state,
            'country': country,
            'experience': experience,
            'tenure': tenure,
            'tag': tag,
            'salary': salary,
            'stock': stock,
            'bonus': bonus,
            'gender': gender,
            'city_id': city_id,
            'dma_id': dma_id,
            'ms_deg': ms_deg,
            'bs_deg': bs_deg,
            'phd_deg': phd_deg,
            'hs': hs,
            'college': college,
            'race': race,
        })
    return salaryAllData


In [7]:
# Reparse data from CSV so we don't mess up above data
salaryRaw = []
for l in readCSV("Levels_Fyi_Salary_Data.csv"):
    salaryRaw.append(l)

salaryAllData = parseSalaryRaw(salaryRaw)
salaryAllData[0]['id']

1

In [8]:
# Parse raw data
salaryAllData = parseSalaryRaw(salaryRaw)

In [9]:
#####
# Training, Validation, and Test sets
#####

# shuffle data
shuffledAllData = sklearn.utils.shuffle(salaryAllData)
shuffledAllData[0]['id']

# create 80 / 10 / 10 : Train / Validation / Test sets
X = shuffledAllData
y = [d['salary'] for d in shuffledAllData]

X_train, X_temp, y_train, y_temp = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=1)
X_valid, X_test, y_valid, y_test = sklearn.model_selection.train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Validate that sets were created with propper sizes
print(str(len(X_train)) + ", " + str(len(y_train)))
print(str(len(X_valid)) + ", " + str(len(y_valid)))
print(str(len(X_test)) + ", " + str(len(y_test)))

# Validate all ids are different
print(str(X_train[0]['id']) + ", " + str(X_valid[0]['id']) + ", " + str(X_test[0]['id']))

47996, 47996
6000, 6000
6000, 6000
12667, 52127, 38935


In [10]:
#####
# Baselines
#   1. Predict average salary in dataset
#   2. Predict average salary for given Years of Experience
#####

In [11]:
# Create some useful collections for these baseline

# create YOE to Average Map based on training data
# We create two dictionaries to track the total salary seen at each YOE
salByExpMap = defaultdict(int)

# And the number of salaries seen at that YOE
entriesByExpMap = defaultdict(int)

# For each entry increment the total sal and num entries at that YOE
# (This could be done with a running avg technique but that is too much work to code)
for x in X_train:
    exp, sal = float(x['experience']), float(x['salary'])
    exp = math.ceil(exp)
    
    total = 0
    numEntries = 0
    
    if salByExpMap[exp]:
        total = salByExpMap[exp]
    if entriesByExpMap[exp]:
        numEntries = entriesByExpMap[exp]
    
    total += sal
    numEntries += 1
    
    salByExpMap[exp] = total
    entriesByExpMap[exp] = numEntries
    

    
# Calculate averages by YOE as well as total avg
totalSal = 0
totalCount = 0

averageSalaryByYoeMap = defaultdict(int)
for exp in entriesByExpMap:
    avg = salByExpMap[exp] / entriesByExpMap[exp]
    averageSalaryByYoeMap[exp] = avg
    
    totalSal += salByExpMap[exp]
    totalCount += entriesByExpMap[exp]

avgSal = totalSal / totalCount

print("Average salary: " + str(avgSal))


def pred_baseline1(x):    
    return avgSal
    
def pred_baseline2(x):    
    exp = math.ceil(float(x['experience']))

    if averageSalaryByYoeMap[exp]:
        return averageSalaryByYoeMap[exp]
    else:
        return avgSal

Average salary: 141891.7676056338


In [12]:
def baseline1():
    # Returns average salary in dataset
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline1(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((pred - actual) / pred))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [13]:
mse_b1, avg_err_b1, avg_percent_err_b1 = baseline1()
print(mse_b1)
print(avg_err_b1)
print(avg_percent_err_b1)

3197621964.2771955
40485.60668309859
28.532738273880234


In [14]:
def baseline2():
    # Takes rounded YOE for each datapoint and predicts average salary for that YOE
    # If there are no entries for that YOE, predict overall average
    y_b1 = []
    y_b1_pred = []
    
    avg_err = 0
    avg_percent_err = 0
    
    for d in X_valid:
        pred = pred_baseline2(d)
        actual = d['salary']
        
        y_b1_pred.append(pred)
        y_b1.append(actual)
        
        avg_err += abs(actual - pred)
        avg_percent_err += abs(100 * ((pred - actual) / pred))
        
    mse = MSE(y_b1_pred, y_b1)
    
    avg_err = avg_err / len(X_valid)
    avg_percent_err = avg_percent_err / len(X_valid)
    
    return mse, avg_err, avg_percent_err

In [15]:
mse_b2, avg_err_b2, avg_percent_err_b2 = baseline2()
print(mse_b2)
print(avg_err_b2)
print(avg_percent_err_b2)

2524095364.789271
34894.83337277445
24.62800225341369


In [16]:
#####
# Model
#####

In [17]:
#####
# Provide a description of your model:
#      1. What model are you using
#      2. What information are you trying to incorporate into your model
#      3. How are you building your feature vector?
#      4. What interesting pieces of information did you learn? (i.e. most influential words, popular cities, etc)
#####

# Write description in comments here

In [18]:
# Model code starts here